<h1 align=center><font size = 5>Capstone Project Week3: Segmenting and Clustering Neighborhoods in Toronto</font></h1>

<h2>Description</h2>
In this notebook, first time we will download data from url https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  and extract data from this url to get table of data for Toronto and process following steps 1
Step 1: to noramalize and stard data downloaded Once the data is in a structured format, we can start the analysis to explore and cluster the neighborhoods in the city of Toronto.
Step 2: to cluster data in maps for data of Toronto
Step 3: examine data in clustered

#### Step 1: Download data and normalize data

In [1]:
#install some pre-libarary
!conda install -c conda-forge folium=0.5.0 --yes # For Foursquare API lab
!conda install -c conda-forge geopy --yes # For the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [34]:
#install some pre-libarary
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

import folium # map rendering library
from bs4 import BeautifulSoup
#import lxml
print('All of Libraries imported.')

All of Libraries imported.


Download scrape data from the following Wikipedia page and convert to pandas frame

In [86]:
# download data from url
url_data='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url_data)
#parse data use  BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')
#find table has class css'wikitable sortable'
table=soup.find('table', attrs={'class':'wikitable sortable'})

#convert html to pandas
table_html_string =str(table)
dfs = pd.read_html(table_html_string)
df=dfs[0]
df.head()


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Preprocess data to normalize and standdata

In [123]:

# ignore  boroughs is not assigned
df = df.drop(df[(df.Borough == "Not assigned")].index)

# set "Not assigned" Neighborhoods same name as Borough:
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
df.Neighborhood.fillna(df.Borough, inplace=True)

# remove duplicate rows:
df=df.drop_duplicates()

# to combine all neighborhoods that has same post code
df_data = pd.DataFrame({'Postcode':df.Postcode.unique()})
#for Neighborhood
df_data['Neighborhood']=pd.Series(list(set(df['Neighborhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df_data.iterrows())
df_data['Neighborhood']=df_data['Neighborhood'].apply(lambda x: ', '.join(x))
# for Borough
df_data['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df_data.iterrows())
df_data.dtypes

df_data.head(10)


,Postcode,Neighborhood,Borough
0,M3A,Parkwoods,North York
1,M4A,Victoria Village,North York
2,M5A,Harbourfront,Downtown Toronto
3,M6A,"Lawrence Manor, Lawrence Heights",North York
4,M7A,Queen's Park,Downtown Toronto
5,M9A,Queen's Park,Queen's Park
6,M1B,"Malvern, Rouge",Scarborough
7,M3B,Don Mills North,North York
8,M4B,"Parkview Hill, Woodbine Gardens",East York
9,M5B,"Ryerson, Garden District",Downtown Toronto
